In [ ]:
import os
from dotenv import load_dotenv

from agents import Agent, WebSearchTool
from agents import ModelSettings, trace, Runner

from pydantic import BaseModel, Field

from IPython.display import display, Markdown

In [ ]:
load_dotenv(override=True)

INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model='gpt-4o-mini',
    model_settings=ModelSettings(tool_choice="required")
)

In [ ]:
message = "Latest battery technology in 2025"

with trace("Search Battery Technology"):
    result = await Runner.run(starting_agent=search_agent, input=message)

display(Markdown(result.final_output))

In [ ]:
# WebSearchPlanner
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
    to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchField(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use fo the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchField] = Field(description = "A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan
)

In [ ]:
message = "Latest battery technology in 2025"

with trace("Search Battery Technology II"):
    result = await Runner.run(starting_agent=planner_agent, input=message)

[non-fatal] Tracing: request failed: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)
[non-fatal] Tracing: max retries reached, giving up on this batch.


In [ ]:
result.final_output

In [ ]:
result.final_output.searches[0].reason

for result in result.final_output.searches:
    print(f"Reason: {result.reason}")
    print(f"Query: {result.query}")
    print("\n\n")